In [1]:
import os
import numpy as np
import keras
from keras.utils import to_categorical
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from keras.layers import Input, Conv2D, BatchNormalization, ReLU, Add, GlobalAveragePooling2D, Dense
from keras.regularizers import l2
from sklearn.model_selection import train_test_split

2023-10-29 09:55:37.748745: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-29 09:55:37.818898: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-29 09:55:38.222044: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-29 09:55:38.222085: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-29 09:55:38.225050: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [22]:
train_dir = r'./RiceLeafs/train'
test_dir = r'./RiceLeafs/validation'

In [23]:
classes=['Unhealthy', 'Healthy']
indices = range(len(classes))
normal_mapping = dict(zip(classes, indices))
reverse_mapping = dict(zip(indices, classes))

In [24]:
training_data = []
class_index = 0

for file in os.listdir(train_dir):
    path = os.path.join(train_dir, file)
    for im in os.listdir(path):
        image = load_img(os.path.join(path, im), grayscale=False, color_mode='rgb', target_size=(80,80))
        image = img_to_array(image)
        image /= 255
        training_data += [[image,class_index]]
    class_index += 1

In [25]:
test_data = []
class_index = 0

for file in os.listdir(test_dir):
    path = os.path.join(test_dir, file)
    for im in os.listdir(path):
        image = load_img(os.path.join(path, im), grayscale=False, color_mode='rgb', target_size=(80,80))
        image = img_to_array(image)
        image /= 255
        test_data += [[image,class_index]]
    class_index += 1

In [26]:
training_images, training_labels = zip(*training_data)
testing_images, testing_labels = zip(*test_data)

training_labels = np.array(to_categorical(training_labels))
training_images = np.array(training_images)
testing_images = np.array(testing_images)

In [27]:
x_train,x_test,y_train,y_test = train_test_split(training_images,training_labels,test_size=0.2,random_state=44)
y_train = np.array(y_train)
y_test = np.array(y_test)
x_train = np.array(x_train)
x_test = np.array(x_test)
print(y_train.shape)
print(y_test.shape)
print(x_train.shape)
print(x_test.shape)

(1287, 2)
(322, 2)
(1287, 80, 80, 3)
(322, 80, 80, 3)


In [28]:
datagen = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.1,
    fill_mode="nearest",
    # brightness_range=[0.8, 1.2],  # Slight color jitter
    # channel_shift_range=20  # Slight color jitter
)

In [29]:
def residual_block(x, filters, kernel_size=3, stride=1):
    shortcut = x
    x = Conv2D(filters, kernel_size=kernel_size, strides=stride, padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(filters, kernel_size=kernel_size, strides=stride, padding="same")(x)
    x = BatchNormalization()(x)
    x = Add()([shortcut, x])
    x = ReLU()(x)
    return x

input = Input(shape=(80, 80, 3))
x = Conv2D(64, (7, 7), padding="same", strides=2)(input)
x = BatchNormalization()(x)
x = ReLU()(x)

x = residual_block(x, 64)
x = residual_block(x, 64)

x = GlobalAveragePooling2D()(x)
x = Dense(2, activation="softmax")(x)  # Assuming 2 classes for classification

model = keras.Model(inputs=input, outputs=x)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 80, 80, 3)]          0         []                            
                                                                                                  
 conv2d_10 (Conv2D)          (None, 40, 40, 64)           9472      ['input_3[0][0]']             
                                                                                                  
 batch_normalization_10 (Ba  (None, 40, 40, 64)           256       ['conv2d_10[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 re_lu_10 (ReLU)             (None, 40, 40, 64)           0         ['batch_normalization_10

In [30]:
print(training_labels)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [31]:
his=model.fit(datagen.flow(x_train,y_train,batch_size=32),validation_data=(x_test,y_test),epochs=5)

Epoch 1/5
41/41 [==============================] - 19s 386ms/step - loss: 0.2583 - accuracy: 0.9044 - val_loss: 0.5280 - val_accuracy: 0.7547
Epoch 2/5
41/41 [==============================] - 15s 368ms/step - loss: 0.2202 - accuracy: 0.9138 - val_loss: 2.7713 - val_accuracy: 0.2484
Epoch 3/5
41/41 [==============================] - 15s 356ms/step - loss: 0.2067 - accuracy: 0.9254 - val_loss: 1.5339 - val_accuracy: 0.2484
Epoch 4/5
41/41 [==============================] - 15s 356ms/step - loss: 0.2000 - accuracy: 0.9316 - val_loss: 2.6570 - val_accuracy: 0.2484
Epoch 5/5
41/41 [==============================] - 15s 354ms/step - loss: 0.2025 - accuracy: 0.9293 - val_loss: 2.4650 - val_accuracy: 0.2484


In [32]:
model.save("resnet90.h5")

/home/harshvardhanpandey/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
